In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pickle

# Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')

# Load dữ liệu
data = pd.read_csv('./train-data/OANDA_XAUUSD_Historical.csv')

# Hàm tạo dataset dạng sliding window với target đã có sẵn (không dùng scaler)
def create_multifeature_dataset(df):
    steps = []
    for lag in range(0, 15):
        step = np.stack([
            df[f'body_{lag}'].values,
            df[f'shadow_top_{lag}'].values,
            df[f'shadow_bottom_{lag}'].values
        ], axis=1)  # Shape: (num_samples, 3)
        steps.append(step)

    # Stack các bước theo chiều time để có shape (num_samples, 15, 3)
    X = np.stack(steps, axis=1)
    y = df['target'].values
    return X, y

X, y = create_multifeature_dataset(data)

# Không cần reshape vì X đã có shape (num_samples, 15, 3)
# X = X.reshape((X.shape[0], 1, X.shape[1]))

# Chia dữ liệu thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình LSTM cho bài toán phân loại nhị phân
model = Sequential([
    Input(shape=(15, 3)),
    LSTM(400, return_sequences=True),
    LSTM(200, return_sequences=False),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Dùng hàm kích hoạt sigmoid cho đầu ra nhị phân
])

# Compile model với loss cho phân loại nhị phân
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Định nghĩa callback dừng sớm nếu val_precision không cải thiện
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Huấn luyện mô hình
model.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    batch_size=64, 
    verbose=1, 
    epochs=10, 
    callbacks=[early_stopping]
)

# Lưu model (không lưu scaler vì không sử dụng)
model.save('./model/predict_model_XAUUSD_v4.h5')


Epoch 1/10
23484/63655 ━━━━━━━━━━━━━━━━━━━━ 26:35 40ms/step - accuracy: 0.5703 - loss: 0.6778

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import os
import shutil
os.environ['TF_DATA_DISABLE_CACHE_LOCKING'] = '1'
cache_dir = "D:/tmp/FOREX_cache"
os.makedirs(cache_dir, exist_ok=True)

# Đường dẫn đến file CSV
file_path = './train-data/FOREX.csv'
batch_size = 64

# 1. Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')
target_column = 'target'
all_columns = columns + [target_column]

# 2. Tạo dataset streaming từ CSV (không load toàn bộ vào bộ nhớ)
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    shuffle=True,
    num_epochs=1,
).ignore_errors()

# 3. Hàm xử lý mỗi batch: sắp xếp lại thứ tự các cột và reshape cho LSTM
def parse_csv(batch):
    # Lấy các cột features theo thứ tự mong muốn
    features = [batch[col] for col in columns]  # mỗi tensor có shape (batch_size,)
    X = tf.stack(features, axis=1)              # shape: (batch_size, num_features)
    X = tf.expand_dims(X, axis=1)                 # reshape thành (batch_size, time_steps=1, num_features)
    y = batch[target_column]
    return X, y

# dataset = raw_dataset.map(parse_csv, num_parallel_calls=tf.data.AUTOTUNE).cache(cache_dir).prefetch(tf.data.AUTOTUNE)
dataset = raw_dataset.map(parse_csv, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

# 4. Unbatch và enumerate để xử lý từng mẫu riêng lẻ, từ đó chia theo tỷ lệ cố định
dataset = dataset.unbatch()       # Tách ra thành từng mẫu đơn
dataset = dataset.enumerate()     # Gán chỉ số cho mỗi mẫu: (index, (X, y))

# 5. Chia dữ liệu theo modulo (ví dụ: 80% train, 20% validation)
def is_train(idx, data):
    # Với mỗi 5 mẫu, 4 mẫu đầu cho train (idx mod 5 < 4)
    return tf.math.mod(idx, 5) < 4

def is_val(idx, data):
    return tf.math.mod(idx, 5) == 4

train_dataset = dataset.filter(is_train).map(lambda idx, data: data)
val_dataset   = dataset.filter(is_val).map(lambda idx, data: data)

# (Tùy chọn) Nếu cần lặp lại dataset vô hạn:
train_dataset = train_dataset.batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)
val_dataset   = val_dataset.batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)

# 7. Tính số bước (steps) dựa trên số dòng trong file CSV
# Đếm số dòng trong file (trừ header)
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1

# Vì ta chia theo tỉ lệ 80/20, ta tính số steps cho train và validation
total_steps = total_samples // batch_size
train_steps = int(0.8 * total_steps)
validation_steps = int(0.2 * total_steps)

# 8. Xây dựng mô hình LSTM
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(1, len(columns))),
    tf.keras.layers.LSTM(400, return_sequences=True),
    tf.keras.layers.LSTM(200, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision()])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # Sử dụng 'val_loss' để đảm bảo metric có giá trị ngay từ epoch đầu
    patience=3,
    restore_best_weights=True
)

# 9. Huấn luyện mô hình
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=50,
          steps_per_epoch=train_steps,
          validation_steps=validation_steps,
          callbacks=[early_stopping],
          verbose=1)

# 10. Sau khi training, lưu model
model.save('./predict_model_XAUUSD_v2.h5')


In [ ]:
import tensorflow as tf

# Đường dẫn đến file CSV
file_path = './train-data/OANDA_XAUUSD_Historical.csv'
batch_size = 64

# 1. Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')
target_column = 'target'
all_columns = columns + [target_column]

# 2. Tạo dataset streaming từ CSV (không load toàn bộ vào bộ nhớ)
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    # shuffle=True,
    num_epochs=1,
).ignore_errors()

# 3. Hàm xử lý mỗi batch: sắp xếp lại thứ tự các cột và reshape cho LSTM
def parse_csv(batch):
    steps = []
    # Lặp qua các lag từ 0 đến 14 để tạo ra 15 time step
    for lag in range(0, 15):
        # Mỗi time step có 3 đặc trưng: body, shadow_top, shadow_bottom
        step = tf.stack([
            batch[f'body_{lag}'],
            batch[f'shadow_top_{lag}'],
            batch[f'shadow_bottom_{lag}']
        ], axis=1)  # Kết quả: shape (batch_size, 3)
        steps.append(step)
    # Stack các time step lại theo chiều time: kết quả shape (batch_size, 15, 3)
    X = tf.stack(steps, axis=1)
    y = batch[target_column]
    return X, y


dataset = raw_dataset.map(parse_csv).prefetch(tf.data.AUTOTUNE)


# 4. Unbatch và enumerate để xử lý từng mẫu riêng lẻ, từ đó chia theo tỷ lệ cố định
dataset = dataset.unbatch()       # Tách ra thành từng mẫu đơn
dataset = dataset.enumerate()     # Gán chỉ số cho mỗi mẫu: (index, (X, y))

# 5. Chia dữ liệu theo modulo (ví dụ: 80% train, 20% validation)
def is_train(idx, data):
    # Với mỗi 5 mẫu, 4 mẫu đầu cho train (idx mod 5 < 4)
    return tf.math.mod(idx, 5) < 4

def is_val(idx, data):
    return tf.math.mod(idx, 5) == 4

train_dataset = dataset.filter(is_train).map(lambda idx, data: data)
val_dataset   = dataset.filter(is_val).map(lambda idx, data: data)

# 6. Re-batch lại dữ liệu
train_dataset = train_dataset.batch(batch_size)
val_dataset   = val_dataset.batch(batch_size)

# (Tùy chọn) Nếu cần lặp lại dataset vô hạn:
train_dataset = train_dataset.repeat()
val_dataset   = val_dataset.repeat()

# 7. Tính số bước (steps) dựa trên số dòng trong file CSV
# Đếm số dòng trong file (trừ header)
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1

# Vì ta chia theo tỉ lệ 80/20, ta tính số steps cho train và validation
total_steps = total_samples // batch_size
train_steps = int(0.8 * total_steps)
validation_steps = int(0.2 * total_steps)

# 8. Xây dựng mô hình LST
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(15, 3)),
    tf.keras.layers.LSTM(400, return_sequences=True),
    tf.keras.layers.LSTM(200, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision()])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # Sử dụng 'val_loss' để đảm bảo metric có giá trị ngay từ epoch đầu
    patience=3,
    restore_best_weights=True
)

# 9. Huấn luyện mô hình
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=50,
          steps_per_epoch=train_steps,
          validation_steps=validation_steps,
          callbacks=[early_stopping],
          verbose=1)

# 10. Sau khi training, lưu model
model.save('./predict_model_XAUUSD_v3.h5')


# Tối ưu 

In [ ]:
import tensorflow as tf
import os
import shutil
os.environ['TF_DATA_DISABLE_CACHE_LOCKING'] = '1'
cache_dir = "D:/tmp/FOREX_cache"
os.makedirs(cache_dir, exist_ok=True)

# Đường dẫn đến file CSV
file_path = './train-data/FOREX.csv'
batch_size = 64

# 1. Xây dựng danh sách các cột features với các lag
lags = range(15)
columns = [f'{col}_{lag}' for col in ['body', 'shadow_top', 'shadow_bottom'] for lag in lags]
target_column = 'target'
all_columns = columns + [target_column]

# 2. Tạo dataset streaming từ CSV
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    shuffle=True,
    num_epochs=1,
).ignore_errors()

# 3. Hàm xử lý mỗi batch: sắp xếp lại thứ tự các cột và reshape cho LSTM
def parse_csv(batch):
    steps = []
    for lag in range(15):
        step = tf.stack([
            batch[f'body_{lag}'],
            batch[f'shadow_top_{lag}'],
            batch[f'shadow_bottom_{lag}']
        ], axis=1)  # (batch_size, 3)
        steps.append(step)
    X = tf.stack(steps, axis=1)  # (batch_size, 15, 3)
    y = batch[target_column]
    return X, y

# Sử dụng parallel map
try:
    dataset = raw_dataset.map(parse_csv, num_parallel_calls=tf.data.AUTOTUNE) \
                         .cache(cache_dir) \
                         .prefetch(tf.data.AUTOTUNE)
    # Thử duyệt qua một vài phần tử để kích hoạt cache.
    for _ in dataset.take(1):
        pass
except tf.errors.DataLossError as e:
    print("Cache bị lỗi, xóa cache và tạo lại...")
    shutil.rmtree(cache_dir)
    os.makedirs(cache_dir, exist_ok=True)
    dataset = raw_dataset.map(parse_csv, num_parallel_calls=tf.data.AUTOTUNE) \
                         .cache(cache_dir) \
                         .prefetch(tf.data.AUTOTUNE)

# Prefetch để giảm latency
dataset = dataset.prefetch(tf.data.AUTOTUNE)

# 4. Chuyển đổi thành từng mẫu đơn
dataset = dataset.unbatch()

# 5. Chia dữ liệu sử dụng shard (thay cho enumerate+filter)
# Giả sử chia thành 5 shard: 80% cho train (shard 0,1,2,3) và 20% cho validation (shard 4)
# Phương án này đòi hỏi dữ liệu đã được xáo trộn
train_shards = [dataset.shard(num_shards=5, index=i) for i in range(4)]
train_dataset = train_shards[0]
for shard in train_shards[1:]:
    train_dataset = train_dataset.concatenate(shard)

val_dataset = dataset.shard(num_shards=5, index=4)

# 6. Re-batch, repeat và prefetch
train_dataset = train_dataset.batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)
val_dataset   = val_dataset.batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)

# 7. Tính số bước (nếu cần tính toán bước dựa trên số dòng)
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1
total_steps = total_samples // batch_size
train_steps = int(0.8 * total_steps)
validation_steps = int(0.2 * total_steps)

# 8. Xây dựng mô hình LSTM
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(15, 3)),
    tf.keras.layers.LSTM(400, return_sequences=True),
    tf.keras.layers.LSTM(200, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision()])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# 9. Huấn luyện mô hình
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=50,
          steps_per_epoch=train_steps,
          validation_steps=validation_steps,
          callbacks=[early_stopping],
          verbose=1)

# 10. Lưu model
model.save('./predict_model_XAUUSD_v3.h5')


In [ ]:
import tensorflow as tf

# Đường dẫn đến file CSV
file_path = './train-data/FOREX.csv'
batch_size = 64

# 1. Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')
target_column = 'target'
all_columns = columns + [target_column]

# 2. Tạo dataset streaming từ CSV (không load toàn bộ vào bộ nhớ)
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    shuffle=True,
    num_epochs=1,
).ignore_errors()

# 3. Hàm xử lý mỗi batch: sắp xếp lại thứ tự các cột và reshape cho LSTM
def parse_csv(batch):
    # Lấy các cột features theo thứ tự mong muốn
    features = [batch[col] for col in columns]  # mỗi tensor có shape (batch_size,)
    X = tf.stack(features, axis=1)              # shape: (batch_size, num_features)
    X = tf.expand_dims(X, axis=1)                 # reshape thành (batch_size, time_steps=1, num_features)
    y = batch[target_column]
    return X, y

dataset = raw_dataset.map(parse_csv)

# 4. Unbatch và enumerate để xử lý từng mẫu riêng lẻ, từ đó chia theo tỷ lệ cố định
dataset = dataset.unbatch()       # Tách ra thành từng mẫu đơn
dataset = dataset.enumerate()     # Gán chỉ số cho mỗi mẫu: (index, (X, y))

# 5. Chia dữ liệu theo modulo (ví dụ: 80% train, 20% validation)
def is_train(idx, data):
    # Với mỗi 5 mẫu, 4 mẫu đầu cho train (idx mod 5 < 4)
    return tf.math.mod(idx, 5) < 4

def is_val(idx, data):
    return tf.math.mod(idx, 5) == 4

train_dataset = dataset.filter(is_train).map(lambda idx, data: data)
val_dataset   = dataset.filter(is_val).map(lambda idx, data: data)

# 6. Re-batch lại dữ liệu
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset   = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# (Tùy chọn) Nếu cần lặp lại dataset vô hạn:
train_dataset = train_dataset.repeat()
val_dataset   = val_dataset.repeat()

# 7. Tính số bước (steps) dựa trên số dòng trong file CSV
# Đếm số dòng trong file (trừ header)
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1

# Vì ta chia theo tỉ lệ 80/20, ta tính số steps cho train và validation
total_steps = total_samples // batch_size
train_steps = int(0.8 * total_steps)
validation_steps = int(0.2 * total_steps)

# 8. Xây dựng mô hình MLP
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(1, len(columns))),
    tf.keras.layers.Flatten(),  # chuyển đổi từ (batch_size, 1, num_features) -> (batch_size, num_features)
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision()])


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Sử dụng 'val_loss' để đảm bảo metric có giá trị ngay từ epoch đầu
    patience=3,
    restore_best_weights=True
)

# 9. Huấn luyện mô hình
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=50,
          steps_per_epoch=train_steps,
          validation_steps=validation_steps,
          callbacks=[early_stopping],
          verbose=1)

# 10. Sau khi training, lưu model
model.save('./predict_model_XAUUSD_v3.h5')


In [ ]:
import dask.dataframe as dd

# Đọc file CSV lớn; Dask tự động chia thành nhiều partition
df = dd.read_csv('./train-data/FOREX.csv')

# Xáo trộn toàn bộ dữ liệu (lấy mẫu ngẫu nhiên với frac=1)
df_shuffled = df.sample(frac=1, random_state=42)

# Gom tất cả các partition thành 1 partition
df_single = df_shuffled.repartition(npartitions=1)

# Ghi ra file CSV duy nhất (tham số single_file=True yêu cầu Dask phiên bản hỗ trợ)
df_single.to_csv('FOREX.csv', index=False, single_file=True)


In [ ]:
import tensorflow as tf

# 1. Cài đặt tf.distribute để chạy song song trên GPU
strategy = tf.distribute.MirroredStrategy()  # Sử dụng tất cả các GPU có sẵn

# Đường dẫn đến file CSV và thiết lập batch size
file_path = './train-data/FOREX.csv'
batch_size = 64

# 2. Xây dựng danh sách các cột features với các lag
lags = list(range(15))
def get_columns_for_lag(lag):
    return [f'body_{lag}', f'shadow_top_{lag}', f'shadow_bottom_{lag}']

columns = []
for lag in lags:
    columns.extend(get_columns_for_lag(lag))
target_column = 'target'
all_columns = columns + [target_column]

# 3. Tạo dataset streaming từ CSV (sử dụng CPU)
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    shuffle=True,
    num_epochs=1,
).ignore_errors()

def parse_csv(batch):
    # Xây dựng dữ liệu với shape: (batch_size, time_steps=15, features=3)
    X_steps = []
    for lag in lags:
        step = tf.stack([
            batch[f'body_{lag}'],
            batch[f'shadow_top_{lag}'],
            batch[f'shadow_bottom_{lag}']
        ], axis=1)  # shape: (batch_size, 3)
        X_steps.append(step)
    X = tf.stack(X_steps, axis=1)  # shape: (batch_size, 15, 3)
    y = batch[target_column]
    return X, y

# Áp dụng map để chuyển đổi dữ liệu
dataset = raw_dataset.map(parse_csv, num_parallel_calls=tf.data.AUTOTUNE)


# Tính số lượng batch theo số dòng (nếu cần)
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1
total_steps = total_samples // batch_size
train_batches = int(0.8 * total_steps)
val_batches = total_steps - train_batches

# Phân chia tập train và validation bằng take và skip
train_dataset = dataset.take(train_batches).repeat().prefetch(tf.data.AUTOTUNE)
val_dataset   = dataset.skip(train_batches).take(val_batches).repeat().prefetch(tf.data.AUTOTUNE)

# 4. Xây dựng mô hình Transformer trong phạm vi của strategy
with strategy.scope():
    # Tham số mô hình
    time_steps = len(lags)       # 15
    num_features = 3             # mỗi time step có 3 features
    embedding_dim = 64           # chiều embedding cho mỗi time step
    num_transformer_blocks = 2   # số Transformer Encoder block
    head_size = 64
    num_heads = 4
    ff_dim = 128
    dropout_rate = 0.1

    # Lớp Positional Embedding
    class PositionalEmbedding(tf.keras.layers.Layer):
        def __init__(self, sequence_length, output_dim):
            super(PositionalEmbedding, self).__init__()
            self.token_proj = tf.keras.layers.Dense(output_dim)
            self.position_embedding = tf.keras.layers.Embedding(
                input_dim=sequence_length, output_dim=output_dim
            )
            self.sequence_length = sequence_length

        def call(self, inputs):
            positions = tf.range(start=0, limit=self.sequence_length, delta=1)
            pos_embed = self.position_embedding(positions)  # (sequence_length, output_dim)
            token_embed = self.token_proj(inputs)           # (batch_size, sequence_length, output_dim)
            return token_embed + pos_embed

    # Định nghĩa Transformer Encoder block
    def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout):
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        attn_output = tf.keras.layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
        )(x, x)
        x = tf.keras.layers.Add()([attn_output, inputs])
        x_ff = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x_ff = tf.keras.layers.Dense(ff_dim, activation="relu")(x_ff)
        x_ff = tf.keras.layers.Dropout(dropout)(x_ff)
        x_ff = tf.keras.layers.Dense(inputs.shape[-1])(x_ff)
        return tf.keras.layers.Add()([x, x_ff])

    # Xây dựng mô hình
    inputs = tf.keras.layers.Input(shape=(time_steps, num_features))
    x = PositionalEmbedding(sequence_length=time_steps, output_dim=embedding_dim)(inputs)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout_rate)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision()])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_precision',
    patience=3,
    restore_best_weights=True
)

# 5. Huấn luyện mô hình
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=1,
          steps_per_epoch=train_batches,
          validation_steps=val_batches,
          callbacks=[early_stopping],
          verbose=1)

# 6. Lưu mô hình sau training
model.save('./predict_model_XAUUSD_transformer_v2.h5')


In [ ]:
import tensorflow as tf

# Đường dẫn đến file CSV
file_path = './train-data/FOREX.csv'
batch_size = 64

# 1. Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = [f'{col}_{lag}' for col in ['body', 'shadow_top', 'shadow_bottom'] for lag in lags]
target_column = 'target'
all_columns = columns + [target_column]

# 2. Tạo dataset streaming từ CSV với đọc song song
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    shuffle=True,
    num_epochs=1,
    num_parallel_reads=tf.data.experimental.AUTOTUNE
).ignore_errors()

# 3. Hàm xử lý mỗi batch: sắp xếp lại thứ tự các cột và reshape cho LSTM
def parse_csv(batch):
    steps = []
    for lag in range(0, 15):
        step = tf.stack([
            batch[f'body_{lag}'],
            batch[f'shadow_top_{lag}'],
            batch[f'shadow_bottom_{lag}']
        ], axis=1)  # Kết quả shape: (batch_size, 3)
        steps.append(step)
    # Stack theo chiều thời gian: (batch_size, 15, 3)
    X = tf.stack(steps, axis=1)
    y = batch[target_column]
    return X, y

# Áp dụng map với xử lý song song
dataset = raw_dataset.map(parse_csv, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# 4. Tùy chọn: Nếu dữ liệu đủ nhỏ để cache, có thể cache sau map
# dataset = dataset.cache()

# 5. Tách dữ liệu thành các mẫu riêng lẻ và sử dụng shard để chia train/validation
dataset = dataset.unbatch()

# Sử dụng shard: Chia dữ liệu thành 5 phần, 4 phần train, 1 phần validation
train_dataset = dataset.shard(num_shards=5, index=0)
for i in range(1, 4):
    train_dataset = train_dataset.concatenate(dataset.shard(num_shards=5, index=i))
val_dataset = dataset.shard(num_shards=5, index=4)

# 6. Re-batch lại dữ liệu và áp dụng prefetch
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset   = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# (Tùy chọn) Nếu cần lặp lại dataset vô hạn:
train_dataset = train_dataset.repeat()
val_dataset   = val_dataset.repeat()

# 7. Tính số bước (steps) dựa trên số dòng trong file CSV
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1  # Trừ header
total_steps = total_samples // batch_size
train_steps = int(0.8 * total_steps)
validation_steps = int(0.2 * total_steps)

# 8. Xây dựng mô hình LSTM
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(15, 3)),
    tf.keras.layers.LSTM(400, return_sequences=True),
    tf.keras.layers.LSTM(200, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision()]
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# 9. Huấn luyện mô hình
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    steps_per_epoch=train_steps,
    validation_steps=validation_steps,
    callbacks=[early_stopping],
    verbose=1
)

# 10. Lưu mô hình sau khi training
model.save('./predict_model_XAUUSD_v3.h5')
